In [16]:
import pandas as pd
import numpy as np
from pathlib import Path
import lightgbm as lgb
import optuna

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score


In [17]:
PATH = Path("merged_libur_cuaca_ispu_ndvi.csv")
df = pd.read_csv(PATH, sep=";")

df["tanggal"] = pd.to_datetime(df["tanggal"], dayfirst=True)
df = df.sort_values(["lokasi_clean", "tanggal"]).reset_index(drop=True)

DROP_COLS = ["max", "parameter_pencemar_kritis", "time", "id", "stasiun"]
df = df.drop(columns=DROP_COLS, errors="ignore")

LABEL_MAP = {"BAIK": 0, "SEDANG": 1, "TIDAK SEHAT": 2}
df = df[df["kategori"].notna()].copy()
df["target"] = df["kategori"].map(LABEL_MAP).astype(int)


In [18]:
# 3. REINDEX HARIAN

def reindex_daily(g):
    idx = pd.date_range(g["tanggal"].min(), g["tanggal"].max(), freq="D")
    g = g.set_index("tanggal").reindex(idx)
    g["lokasi_clean"] = g["lokasi_clean"].iloc[0]
    return g.reset_index().rename(columns={"index": "tanggal"})

df = df.groupby("lokasi_clean", group_keys=False).apply(reindex_daily)

# 4. HANDLE PM2.5 MISSING
df["pm25_missing"] = df["pm_duakomalima"].isna().astype(int)

median_pm25 = (
    df[df["tanggal"] >= "2021-01-01"]
    .groupby("lokasi_clean")["pm_duakomalima"]
    .median()
)

df["pm_duakomalima"] = df["pm_duakomalima"].fillna(
    df["lokasi_clean"].map(median_pm25)
)

df = df[df["target"].notna()].copy()


# 5. TIME FEATURES
df["month"] = df["tanggal"].dt.month
df["dayofyear"] = df["tanggal"].dt.dayofyear
df["dayofweek"] = df["tanggal"].dt.dayofweek

df["month_sin"] = np.sin(2*np.pi*df["month"]/12)
df["month_cos"] = np.cos(2*np.pi*df["month"]/12)

df["doy_sin"] = np.sin(2*np.pi*df["dayofyear"]/365)
df["doy_cos"] = np.cos(2*np.pi*df["dayofyear"]/365)

df["dow_sin"] = np.sin(2*np.pi*df["dayofweek"]/7)
df["dow_cos"] = np.cos(2*np.pi*df["dayofweek"]/7)


# 6. DEFINE COLS
POLLUTANT_COLS = [
    "pm_sepuluh","pm_duakomalima","ozon",
    "nitrogen_dioksida","sulfur_dioksida","karbon_monoksida"
]

# 7. BUILD FORECAST DATASET
def build_forecast_df(df_source, H=30):
    rows = []

    for loc, g in df_source.groupby("lokasi_clean"):
        g = g.sort_values("tanggal").reset_index(drop=True)

        for col in POLLUTANT_COLS:
            g[f"{col}_lag1"] = g[col].shift(0)
            g[f"{col}_lag2"] = g[col].shift(1)
            g[f"{col}_lag3"] = g[col].shift(2)

        for h in range(1, H+1):
            g_future = g.shift(-h)

            temp = pd.DataFrame({
                "lokasi_clean": loc,
                "tanggal": g["tanggal"],
                "horizon": h,
                "month": g["tanggal"].dt.month,
                "dayofyear": g["tanggal"].dt.dayofyear,
                "dayofweek": g["tanggal"].dt.dayofweek,
            })

            for col in POLLUTANT_COLS:
                temp[f"{col}_lag1"] = g[f"{col}_lag1"]
                temp[f"{col}_lag2"] = g[f"{col}_lag2"]
                temp[f"{col}_lag3"] = g[f"{col}_lag3"]
                temp[f"target_{col}"] = g_future[col]

            rows.append(temp)

    return pd.concat(rows, ignore_index=True).dropna().reset_index(drop=True)


df_model = df.reset_index(drop=True)
df_forecast = build_forecast_df(df_model)

# 8. ENCODE LOCATION
le_loc = LabelEncoder()
df_model["lokasi_enc"] = le_loc.fit_transform(df_model["lokasi_clean"])
df_forecast["lokasi_enc"] = le_loc.transform(df_forecast["lokasi_clean"])





C:\Users\USER\AppData\Local\Temp\ipykernel_28280\1669613731.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("lokasi_clean", group_keys=False).apply(reindex_daily)


In [19]:
# 9. TIME SERIES SPLIT
SPLIT_DATE = "2024-01-01"

df_forecast_train = df_forecast[df_forecast["tanggal"] < SPLIT_DATE]
df_forecast_val   = df_forecast[df_forecast["tanggal"] >= SPLIT_DATE]


In [20]:
# 10. TRAIN FORECAST MODELS (EARLY STOPPING)
pollutant_models = {}

FEATURES_F = (
    [f"{c}_lag1" for c in POLLUTANT_COLS] +
    [f"{c}_lag2" for c in POLLUTANT_COLS] +
    [f"{c}_lag3" for c in POLLUTANT_COLS] +
    ["month","dayofyear","dayofweek","lokasi_enc","horizon"]
)

for col in POLLUTANT_COLS:

    y_col = f"target_{col}"

    train_data = lgb.Dataset(
        df_forecast_train[FEATURES_F],
        label=df_forecast_train[y_col]
    )

    val_data = lgb.Dataset(
        df_forecast_val[FEATURES_F],
        label=df_forecast_val[y_col]
    )

    model = lgb.train(
        dict(objective="regression", metric="l2", learning_rate=0.05, seed=42),
        train_data,
        num_boost_round=2000,
        valid_sets=[val_data],
        callbacks=[lgb.early_stopping(100), lgb.log_evaluation(0)]
    )

    pollutant_models[col] = model


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[146]	valid_0's l2: 190.877
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[159]	valid_0's l2: 420.382
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[127]	valid_0's l2: 111.444
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[59]	valid_0's l2: 265.372
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[84]	valid_0's l2: 98.1821
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[46]	valid_0's l2: 34.3865


In [21]:
# 11. BUILD ISPU LAG
for col in POLLUTANT_COLS:
    for lag in [1,2,3]:
        df_model[f"{col}_lag{lag}"] = df_model.groupby("lokasi_clean")[col].shift(lag)

df_model = df_model.dropna().reset_index(drop=True)

# 12. SPLIT ISPU
train_df = df_model[df_model["tanggal"] < SPLIT_DATE]
val_df   = df_model[df_model["tanggal"] >= SPLIT_DATE]

# 13. FEATURES ISPU
ISPU_FEATURES = (
    POLLUTANT_COLS +
    [f"{c}_lag{l}" for c in POLLUTANT_COLS for l in [1,2,3]] +
    ["month_sin","month_cos","doy_sin","doy_cos","dow_sin","dow_cos","lokasi_enc"]
)

X_train, y_train = train_df[ISPU_FEATURES], train_df["target"]
X_val,   y_val   = val_df[ISPU_FEATURES],   val_df["target"]


In [22]:
# 14. OPTUNA
def objective(trial):

    params = {
        "objective":"multiclass",
        "num_class":3,
        "metric":"multi_logloss",
        "learning_rate":trial.suggest_float("lr",0.01,0.1,log=True),
        "num_leaves":trial.suggest_int("leaves",31,255),
        "min_data_in_leaf":trial.suggest_int("minleaf",10,80),
        "feature_fraction":trial.suggest_float("ff",0.6,1.0),
        "bagging_fraction":trial.suggest_float("bf",0.6,1.0),
        "bagging_freq":trial.suggest_int("bq",1,7),
        "lambda_l1":trial.suggest_float("l1",0,5),
        "lambda_l2":trial.suggest_float("l2",0,5),
        "verbosity":-1,
        "seed":42
    }

    model = lgb.train(
        params,
        lgb.Dataset(X_train, label=y_train),
        num_boost_round=2000,
        valid_sets=[lgb.Dataset(X_val, label=y_val)],
        callbacks=[lgb.early_stopping(100), lgb.log_evaluation(0)]
    )

    pred = model.predict(X_val).argmax(1)
    return f1_score(y_val, pred, average="macro")


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

best_params = study.best_params


[I 2026-02-07 22:13:01,492] A new study created in memory with name: no-name-d47294d5-f099-4e9a-a722-e6bee0089752


Training until validation scores don't improve for 100 rounds


[I 2026-02-07 22:13:03,643] Trial 0 finished with value: 0.9178569685331691 and parameters: {'lr': 0.052361280051298184, 'leaves': 145, 'minleaf': 60, 'ff': 0.9063781486199081, 'bf': 0.6097957258752016, 'bq': 1, 'l1': 1.3619214960901982, 'l2': 3.3422280184001125}. Best is trial 0 with value: 0.9178569685331691.


Early stopping, best iteration is:
[96]	valid_0's multi_logloss: 0.176381
Training until validation scores don't improve for 100 rounds


[I 2026-02-07 22:13:06,908] Trial 1 finished with value: 0.9106729327178229 and parameters: {'lr': 0.01779701704090814, 'leaves': 152, 'minleaf': 76, 'ff': 0.9620189207602411, 'bf': 0.7961915651264743, 'bq': 2, 'l1': 4.575356389596792, 'l2': 2.005877270741618}. Best is trial 0 with value: 0.9178569685331691.


...

Early stopping, best iteration is:
[258]	valid_0's multi_logloss: 0.158064
Training until validation scores don't improve for 100 rounds


[I 2026-02-07 22:15:19,405] Trial 48 finished with value: 0.9253168350872732 and parameters: {'lr': 0.023812874628638316, 'leaves': 161, 'minleaf': 20, 'ff': 0.9317430284789722, 'bf': 0.7149227022142729, 'bq': 5, 'l1': 4.721773059389308, 'l2': 3.1387936943882218}. Best is trial 16 with value: 0.9290604803464734.


Early stopping, best iteration is:
[205]	valid_0's multi_logloss: 0.166231
Training until validation scores don't improve for 100 rounds


[I 2026-02-07 22:15:22,193] Trial 49 finished with value: 0.9196978202914442 and parameters: {'lr': 0.029363419740075423, 'leaves': 118, 'minleaf': 27, 'ff': 0.8866523414647521, 'bf': 0.8885036235513852, 'bq': 7, 'l1': 4.016238566600025, 'l2': 2.540715773026972}. Best is trial 16 with value: 0.9290604803464734.


Early stopping, best iteration is:
[175]	valid_0's multi_logloss: 0.166632


In [23]:
# 15. TRAIN FINAL MODEL
best_params.update(dict(
    objective="multiclass",
    num_class=3,
    metric="multi_logloss",   # ← JANGAN "None"
    seed=42
))

ispu_model = lgb.train(
    best_params,
    lgb.Dataset(X_train, label=y_train),
    num_boost_round=2000,
    valid_sets=[lgb.Dataset(X_val, label=y_val)],
    callbacks=[lgb.early_stopping(100), lgb.log_evaluation(50)]
)


Training until validation scores don't improve for 100 rounds
[50]	valid_0's multi_logloss: 0.198149
[100]	valid_0's multi_logloss: 0.264381
Early stopping, best iteration is:
[26]	valid_0's multi_logloss: 0.187607


In [24]:
# THRESHOLD
probs_val = ispu_model.predict(X_val)

best_t = [0.33, 0.33, 0.33]

for c in range(3):
    best_f1 = 0
    for t in np.linspace(0.2, 0.6, 25):
        pred = probs_val.argmax(1)
        pred[probs_val[:, c] > t] = c
        f1 = f1_score(y_val, pred, average="macro")

        if f1 > best_f1:
            best_f1 = f1
            best_t[c] = t

print("Best threshold:", best_t)


Best threshold: [0.48333333333333334, 0.21666666666666667, 0.21666666666666667]


In [25]:
# FORECAST TEST
sub = pd.read_csv("sample_submission.csv")

sub["tanggal"] = pd.to_datetime(sub["id"].str.split("_").str[0])
sub["lokasi_clean"] = sub["id"].str.split("_").str[1]
sub = sub.sort_values(["lokasi_clean", "tanggal"]).reset_index(drop=True)

sub["lokasi_enc"] = le_loc.transform(sub["lokasi_clean"])

last_dates = df_model.groupby("lokasi_clean")["tanggal"].max().to_dict()

forecast_rows = []

for loc in sub["lokasi_clean"].unique():

    hist = (
        df_model[df_model["lokasi_clean"] == loc]
        .sort_values("tanggal")
        .iloc[-3:][POLLUTANT_COLS]
        .values.tolist()
    )

    future_rows = sub[sub["lokasi_clean"] == loc].sort_values("tanggal")

    for _, row in future_rows.iterrows():

        tgl = row["tanggal"]

        h = (tgl - last_dates[loc]).days
        h = max(1, min(30, h))

        feat = {
            "horizon": h,
            "month": tgl.month,
            "dayofyear": tgl.dayofyear,
            "dayofweek": tgl.dayofweek,
            "lokasi_enc": row["lokasi_enc"],
        }

        for i, col in enumerate(POLLUTANT_COLS):
            feat[f"{col}_lag1"] = hist[-1][i]
            feat[f"{col}_lag2"] = hist[-2][i]
            feat[f"{col}_lag3"] = hist[-3][i]

        Xf = pd.DataFrame([feat])

        new_vals = []
        preds_pol = {}

        for i, col in enumerate(POLLUTANT_COLS):
            pred = pollutant_models[col].predict(Xf)[0]
            new_vals.append(pred)
            preds_pol[col] = pred

        hist.append(new_vals)
        hist.pop(0)

        feat.update(preds_pol)

        for i, col in enumerate(POLLUTANT_COLS):
            feat[f"{col}_lag1"] = hist[-1][i]
            feat[f"{col}_lag2"] = hist[-2][i]
            feat[f"{col}_lag3"] = hist[-3][i]

        feat["id"] = row["id"]
        forecast_rows.append(feat)

df_future = pd.DataFrame(forecast_rows)


In [26]:
# ===============================
# PREDIKSI ISPU (FINAL FIX)
# ===============================
results = []

for _, row in df_future.iterrows():

    feat_ispu = {col: row[col] for col in POLLUTANT_COLS}

    for col in POLLUTANT_COLS:
        feat_ispu[f"{col}_lag1"] = row[f"{col}_lag1"]
        feat_ispu[f"{col}_lag2"] = row[f"{col}_lag2"]
        feat_ispu[f"{col}_lag3"] = row[f"{col}_lag3"]

    month = row["month"]
    doy   = row["dayofyear"]
    dow   = row["dayofweek"]

    feat_ispu.update({
        "month_sin": np.sin(2*np.pi*month/12),
        "month_cos": np.cos(2*np.pi*month/12),
        "doy_sin": np.sin(2*np.pi*doy/365),
        "doy_cos": np.cos(2*np.pi*doy/365),
        "dow_sin": np.sin(2*np.pi*dow/7),
        "dow_cos": np.cos(2*np.pi*dow/7),
        "lokasi_enc": row["lokasi_enc"],
    })

    Xi = pd.DataFrame([feat_ispu])

    prob = ispu_model.predict(Xi)[0]

    # ===============================
    # THRESHOLD DECISION (BENAR)
    # ===============================
    adj_prob = prob - best_t
    pred = int(np.argmax(adj_prob))
    results.append(pred)


# ===============================
# BUILD SUBMISSION
# ===============================
INV_LABEL_MAP = {0: "BAIK", 1: "SEDANG", 2: "TIDAK SEHAT"}

sub_final = pd.read_csv("sample_submission.csv")

pred_df = df_future[["id"]].copy()
pred_df["kategori"] = [INV_LABEL_MAP[i] for i in results]

sub_final = sub_final.merge(pred_df, on="id", how="left")
sub_final.to_csv("submission_final.csv", index=False)

print("✅ submission_final.csv siap upload")


✅ submission_final.csv siap upload


In [27]:
train_df["target"].value_counts(normalize=True)


target
1.0    0.655833
2.0    0.188702
0.0    0.155465
Name: proportion, dtype: float64

In [28]:
probs_val = ispu_model.predict(X_val)
pred_val = probs_val.argmax(1)

print("Macro-F1 VAL:", f1_score(y_val, pred_val, average="macro"))


Macro-F1 VAL: 0.9012628016693925


In [29]:
probs_val = ispu_model.predict(X_val)
pred_val = probs_val.argmax(1)

print("Macro-F1:", f1_score(y_val, pred_val, average="macro"))
print(pd.Series(pred_val).value_counts(normalize=True))


Macro-F1: 0.9012628016693925
1    0.749917
0    0.152626
2    0.097456
Name: proportion, dtype: float64


In [ ]:
for col in POLLUTANT_COLS:
    print(col, "std =", df_future[col].std())


pm_sepuluh std = 2.2983254802601243
pm_duakomalima std = 0.9432476586281882
ozon std = 7.027937012360093
nitrogen_dioksida std = 0.35573687418532174
sulfur_dioksida std = 0.26740948268988
karbon_monoksida std = 1.0674138541566285


: 